# Timeseries

This section shows how to produce **time-series** from FVCOM–ERSEM output at three scopes:

* **Domain mean** — averages over the whole model domain (area-weighted if `art1` is present).
* **Station** — extracts the nearest model node to each `(name, lat, lon)` and plots a series at that point.
* **Region** — masks nodes **inside** a polygon (shapefile/CSV) and plots a regional aggregate.

Figures are written under:

```
FIG_DIR/<basename(BASE_DIR)>/timeseries/
```

…unless overridden via `FVCOM_PLOT_SUBDIR`.

---

###  Choosing variables, time windows, and depth

* **Variables**
  You can pass either native tracers (e.g., `P1_c`, `temp`) or **group names** you define in `GROUPS` (e.g., `chl`, `phyto`, `zoo`, `DOC`...explined in more detail below). Styles (colours, norms) come from `PLOT_STYLES` when provided.

* **Time selection**

  * `months=[...]` — calendar months across all years (e.g., `[7]` for July or `[4,5,6,7,8,9,10]` for Apr–Oct).
  * `years=[...]` — specific calendar years (e.g., `[2018]` or `[2019,2020]`).
  * `start_date="YYYY-MM-DD", end_date="YYYY-MM-DD"` — explicit date range.

* **Depth selection** (shorthand accepted)

  * `"surface"` / `"bottom"` / `"depth_avg"`
  * **Sigma layer index**: `depth=5` → layer `k=5`
  * **Sigma value**: `depth=-0.7` → sigma `s=-0.7` (in `[-1, 0]`)
  * **Absolute depth (m)**: `depth=-8.0` → `z = −8 m` (downward)

  > Notes: floats in `[-1, 0]` are treated as sigma; other floats are meters. Absolute-depth requires a vertical coordinate with `siglay` (default `z`).

---
### Combining multiple variables, regions, or stations in one plot

You can make **multi-line plots** instead of one image per variable or location by using the keyword:

```python
combine_by="var"      # or "region" or "station"
```
This option controls what is shown as separate lines within the same figure.

combine_by value	   One plot per...	   Lines represent...	    Typical use
    "var"	         region / station	      variables	  Compare multiple tracers at one location
  "region"	            variable	           regions	  Compare regions for a single tracer
  "station"         	variable	          stations	  Compare stations for a single tracer

If you leave `combine_by=None` (the default) or omit it entirely, the script will produce one PNG per (variable × region/station) pair.

###  Scopes & required metadata

* **Domain mean**
  Uses the full mesh (area weighting if `art1` exists). No extra metadata needed.

* **Station**
  Uses your `STATIONS` list of `(name, lat, lon)` in WGS84. The nearest model **node** is chosen by great-circle distance (WGS84 ellipsoid).

  > Reminder: longitudes west of Greenwich are **negative**.

* **Region**
  Uses your `REGIONS` list of `(region_name, spec_dict)`, where `spec_dict` provides **one** of:

  * `{"shapefile": "/path/to/region.shp"}` (optionally with `name_field` / `name_equals`), or
  * `{"csv_boundary": "/path/to/boundary.csv", "lon_col": "lon", "lat_col": "lat", "convex_hull": True}`.
    The polygon is converted to a grid mask; nodes inside are included. If `nv` exists, “strict” element-inclusion may be used. If `art1` exists, regional means are area-weighted.

---

###  File naming & outputs

Output filenames follow a structured pattern so you can tell **scope**, **variable**, **depth**, and **time filter** at a glance, e.g.:

```
<basename(BASE_DIR)>__<Scope>__<VarOrGroup>__<DepthTag>__<TimeLabel>__Timeseries.png
```

* **Scope**: `Domain`, `Station_<NAME>`, or `Region_<NAME>`
* **DepthTag**: `Surface`, `Bottom`, `DepthAvg`, `SigmaK5`, `SigmaS0.7`, `Z8m`, etc.
* **TimeLabel**: derived from your `months`/`years`/`start_date`–`end_date`.

> With your paths, groups, styles, stations, and regions already set, you’re ready to run the time-series cells for domain, station, and region—just choose the variables, time window, and depth as described above.



In [4]:
#Setup

BASE_DIR = "/data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var"
FILE_PATTERN = "erie_00??.nc"
FIG_DIR      = "/data/proteus1/scratch/moja/projects/Lake_Erie/fviz-plots/"


STATIONS = [
    ("WE12", 41.90, -83.10),
    ("WE13", 41.80, -83.20),
]

REGIONS = [
    ("Central", {
        "shapefile": "../data/shapefiles/central_basin_single.shp"
    }),
    ("East", {
        "shapefile": "../data/shapefiles/east_basin_single.shp"
    }),
    ("West", {
        "shapefile": "../data/shapefiles/west_basin_single.shp"
    }),
]

GROUPS = {
    "DOC":   "R1_c + R2_c + R3_c + T1_30d_c + T2_30d_c",  # dissolved organic carbon (sum of pools)
    "phyto": ["P1_c", "P2_c", "P4_c", "P5_c"],            # total phytoplankton carbon (sum)
    "zoo":   ["Z4_c", "Z5_c", "Z6_c"],                    # total zooplankton carbon (sum)
    "chl":   "P1_Chl + P2_Chl + P4_Chl + P5_Chl",         # total chlorophyll (sum)
}

PLOT_STYLES = {
    "temp":   {"line_color": "lightblue", "cmap": "coolwarm"},
    "DOC":   {"line_color": "blue", "cmap": "viridis"},
    "chl":   {"line_color": "lightgreen", "cmap": "Greens", "vmin": 0.0, "vmax": 5.0},
    "phyto": {"line_color": "darkgreen","cmap": "YlGn"},
    "zoo":   {"line_color": "purple","cmap": "PuBu"},
}

# Package imports
from fvcomersemviz.io import load_from_base
from fvcomersemviz.utils import out_dir, file_prefix
from fvcomersemviz.plot import (
    hr, info, bullet, kv,
    try_register_progress_bar,
    list_files, summarize_files,
    plot_call,
    print_dataset_summary,
    ensure_paths_exist,
    sample_output_listing,
)

import matplotlib.pyplot as plt
from IPython.display import display

bullet("\nStations (name, lat, lon):")
for s in STATIONS:
    bullet(f"• {s}")

bullet("\nRegions provided:")
for name, spec in REGIONS:
    bullet(f"• {name}: {spec}")
ensure_paths_exist(REGIONS)

#  Discover files
info(" Discovering files")
files = list_files(BASE_DIR, FILE_PATTERN)
summarize_files(files)
if not files:
    print("\nNo files found. Exiting.")
    sys.exit(2)

#  Load dataset
info(" Loading dataset (this may be lazy if Dask is available)")
ds = load_from_base(BASE_DIR, FILE_PATTERN)
bullet("Dataset loaded. Summary:")
print_dataset_summary(ds)

# Where figures will go / filename prefix
out_folder = out_dir(BASE_DIR, FIG_DIR)
prefix = file_prefix(BASE_DIR)
kv("Figure folder", out_folder)
kv("Filename prefix", prefix)


  
  Stations (name, lat, lon):
  • ('WE12', 41.9, -83.1)
  • ('WE13', 41.8, -83.2)
  
  Regions provided:
  • Central: {'shapefile': '../data/shapefiles/central_basin_single.shp'}
  • East: {'shapefile': '../data/shapefiles/east_basin_single.shp'}
  • West: {'shapefile': '../data/shapefiles/west_basin_single.shp'}

 Discovering files
------------------------------------------------------------------------------
  - Matched files      3
  • /data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var/erie_0001.nc
  • /data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var/erie_0002.nc
  • /data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var/erie_0003.nc

 Loading dataset (this may be lazy if Dask is available)
------------------------------------------------------------------------------
[io] Trying engine='scipy' for open_mfdataset …
[io] Using engine='scipy'.
  Dataset loaded. Summary:
  - Dimensions         {'nele': 11509, 'node': 6106, 'sigla

In [5]:
# --- Timeseries examples: domain, station, region  ---
from fvcomersemviz.plots.timeseries import (
    domain_mean_timeseries,
    station_timeseries,
    region_timeseries,
    domain_three_panel,
    station_three_panel,
    region_three_panel,
)




# 1) Domain mean timeseries 
# Full argument reference for domain_mean_timeseries(...)
# Each parameter below is annotated with what it does and accepted values.

# def domain_mean_timeseries(
#     ds: xr.Dataset,                      # The opened FVCOM–ERSEM dataset (e.g., from load_from_base()).
#     variables: List[str],                # One or more variable names to plot. Each entry can be:
#                                          #   • a native variable in ds (e.g., "temp", "P1_c"), or
#                                          #   • a group name defined in `groups` (e.g., "chl", "DOC").
#     *,
#     depth: Any,                          # Vertical selection for all variables (dataset-level slice unless absolute z):
#                                          #   "surface" | "bottom" | "depth_avg"
#                                          #   int (sigma index, k)          -> e.g., 5
#                                          #   float in [-1, 0] (sigma val)  -> e.g., -0.7
#                                          #   other float (absolute depth m)-> e.g., -8.0 (8 m below surface)
#                                          #   ("z_m", z) or {"z_m": z}      -> explicit absolute-depth form
#     months: Optional[List[int]] = None,  # Filter to calendar months (1–12). Example: [7] for July, [4,5,6,7,8,9,10] for Apr–Oct.
#     years: Optional[List[int]] = None,   # Filter to calendar years. Example: [2018] or [2019, 2020].
#     start_date: Optional[str] = None,    # Start date (inclusive) "YYYY-MM-DD". Use together with end_date.
#     end_date: Optional[str] = None,      # End date (inclusive) "YYYY-MM-DD". Use together with start_date.
#     base_dir: str,                       # Model run directory; used for filename prefix and output folder structure.
#     figures_root: str,                   # Root folder for figures. Module subfolder is auto-added (e.g., /timeseries/).
#     groups: Optional[Dict[str, Any]] = None,  # Composite/group definitions so you can request semantic vars:
#                                               #   "chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"
#                                               #   "phyto": ["P1_c","P2_c","P4_c","P5_c"]  (summed elementwise)
#     linewidth: float = 1.5,              # Line thickness for plotted series.
#     figsize: tuple = (10, 4),            # Figure size in inches (width, height).
#     dpi: int = 150,                      # Output resolution for saved PNGs.
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional per-variable styles, e.g.:
#                                               #   {"temp": {"line_color": "lightblue"},
#                                               #    "chl":  {"line_color": "lightgreen"}}
#                                               # If a var has no style, Matplotlib defaults are used.
#     verbose: bool = True,                # Print progress (selected depth, time window, saved path, etc.).
#     combine_by: Optional[str] = None,    # Multi-line mode:
#                                          #   None      -> one PNG per variable (default).
#                                          #   "var"     -> one PNG total with multiple lines (one per variable).
# ) -> None:
#     """
#     Plot domain-wide mean time series and save PNG(s) to disk.
#
#     File name pattern:
#       <prefix>__Domain__<VarOrMulti>__<DepthTag>__<TimeLabel>__Timeseries[__CombinedByVar].png
#
#     Examples:
#       # Separate figures (one per variable)
#       domain_mean_timeseries(ds, ["DOC", "chl", "temp"], depth="surface",
#                              months=[7], base_dir=BASE_DIR, figures_root=FIG_DIR,
#                              groups=GROUPS, styles=PLOT_STYLES)
#
#       # One multi-line figure (lines = variables)
#       domain_mean_timeseries(ds, ["DOC", "chl", "temp"], depth="surface",
#                              months=[7], base_dir=BASE_DIR, figures_root=FIG_DIR,
#                              groups=GROUPS, styles=PLOT_STYLES, combine_by="var")
#     """


#Example — surface DOC + chl + temp, July (months=[7]) - makes 3 figures (one for each variable)
fig = domain_mean_timeseries(
    ds=ds,
    variables=["DOC", "chl", "temp"],
    depth="surface",
    months=[7],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)

# Domain (surface, Jul), one plot with DOC + chl + temp
domain_mean_timeseries(
    ds=ds,
    variables=["DOC", "chl", "temp"],
    depth="surface",
    months=[7],
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS, styles=PLOT_STYLES,
    combine_by="var",
    verbose=False,
)

# 2) Station timeseries
# def station_timeseries(
#     ds: xr.Dataset,                               # Opened FVCOM–ERSEM dataset (e.g., via load_from_base()).
#     variables: List[str],                         # One or more series to plot. Each can be:
#                                                   #   • a native variable in ds (e.g., "temp", "P1_c"), or
#                                                   #   • a group name from `groups` (e.g., "chl", "DOC", "phyto").
#     stations: List[Tuple[str, float, float]],     # Station list: (name, lat, lon) in WGS84.
#                                                   #   - Longitude west of Greenwich should be negative (e.g., -83.10).
#                                                   #   - The nearest model *node* is found by great-circle distance (WGS84).
#     *,
#     depth: Any,                                   # Vertical selection at each station:
#                                                   #   "surface" | "bottom" | "depth_avg"
#                                                   #   int (sigma index, k)           -> e.g., 5
#                                                   #   float in [-1, 0] (sigma val)   -> e.g., -0.7
#                                                   #   other float (absolute depth m) -> e.g., -8.0 (8 m below surface)
#                                                   #   ("z_m", z) or {"z_m": z}       -> explicit absolute-depth form
#     months: Optional[List[int]] = None,           # Optional month filter (1–12). Example: [4,5,6,7,8,9,10] for Apr–Oct.
#     years: Optional[List[int]] = None,            # Optional year filter. Example: [2018] or [2019, 2020].
#     start_date: Optional[str] = None,             # Optional start date "YYYY-MM-DD" (used with end_date).
#     end_date: Optional[str] = None,               # Optional end date   "YYYY-MM-DD" (used with start_date).
#     base_dir: str,                                # Model run directory; used for filename prefix and output folder structure.
#     figures_root: str,                            # Root output folder. A module subfolder (e.g., /timeseries/) is added automatically.
#     groups: Optional[Dict[str, Any]] = None,      # Composite definitions so you can request semantic variables:
#                                                   #   "chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"
#                                                   #   "phyto": ["P1_c","P2_c","P4_c","P5_c"]   (elementwise sum)
#     linewidth: float = 1.5,                       # Line thickness.
#     figsize: tuple = (10, 4),                     # Figure size in inches (width, height).
#     dpi: int = 150,                               # PNG resolution.
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional per-variable styles, e.g.:
#                                                   #   {"temp": {"line_color": "lightblue"},
#                                                   #    "DOC":  {"line_color": "blue"}}
#     verbose: bool = True,                         # Print progress (resolved node index, time window, saved path, etc.).
#     combine_by: Optional[str] = None,             # Multi-line modes for convenience:
#                                                   #   None       -> one PNG per (station × variable)  [default]
#                                                   #   "var"      -> one PNG per station,  lines = variables
#                                                   #   "station"  -> one PNG per variable, lines = stations
# ) -> None:
#     """

# Notes:
# - Nearest-node lookup uses great-circle distance in WGS84; ensure station lon/lat are WGS84 and lon west < 0.
# - Composites in `groups` allow variables like "chl"/"phyto"/"zoo" without rewriting expressions each time.
# - Works with Dask-chunked datasets; computation is triggered during reduction/plot.
# - Returns None; to view in a notebook, display the saved PNGs afterwards (e.g., with a gallery cell).

#Example — depth-averaged phyto at first station in STATIONS
fig = station_timeseries(
    ds=ds,
    variables=["phyto"],
    stations=[STATIONS[0]],  # e.g., ("WE12", 41.90, -83.10)
    depth="depth_avg",
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)

# Example -Station WE12 — z = -5 m, Apr–Oct 2018: temp + DOC on one plot
fig = station_timeseries(
    ds=ds,
    variables=["temp", "DOC"],
    stations=[STATIONS[0]],                      # e.g., ("WE12", 41.90, -83.10)
    depth=-5.0,                                  # absolute metres below surface (requires vertical coords)
    start_date="2018-04-01", end_date="2018-10-31",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS, styles=PLOT_STYLES,
    combine_by="var",
    verbose=False,
)

# Example - All stations — surface temp, Apr–Oct 2018: one plot, one line per station
fig = station_timeseries(
    ds=ds,
    variables=["temp"],
    stations=STATIONS,                           # multiple stations
    depth="surface",
    start_date="2018-04-01", end_date="2018-10-31",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS, styles=PLOT_STYLES,
    combine_by="station",
    verbose=False,
)


# 3) Region timeseries 
# def region_timeseries(
#     ds: xr.Dataset,                               # Opened FVCOM–ERSEM dataset (e.g., via load_from_base()).
#     variables: List[str],                         # One or more series to plot. Each can be:
#                                                   #   • a native variable in ds (e.g., "temp", "P1_c"), or
#                                                   #   • a group name from `groups` (e.g., "chl", "DOC", "phyto").
#     regions: List[Tuple[str, Dict[str, Any]]],    # Region list as (region_name, spec_dict). spec_dict provides EXACTLY ONE source:
#                                                   #   {"shapefile": "/path/to/region.shp"}                      # optional: "name_field", "name_equals"
#                                                   #   {"csv_boundary": "/path/to/boundary.csv"}                 # optional: "lon_col", "lat_col", "convex_hull"
#     *,
#     depth: Any,                                   # Vertical selection before spatial aggregation:
#                                                   #   "surface" | "bottom" | "depth_avg"
#                                                   #   int (sigma index, k)           -> e.g., 5
#                                                   #   float in [-1, 0] (sigma val)   -> e.g., -0.7
#                                                   #   other float (absolute depth m) -> e.g., -8.0 (8 m below surface)
#                                                   #   ("z_m", z) or {"z_m": z}       -> explicit absolute-depth form
#     months: Optional[List[int]] = None,           # Optional month filter (1–12). Example: [4,5,6,7,8,9,10] for Apr–Oct.
#     years: Optional[List[int]] = None,            # Optional year filter. Example: [2018] or [2019, 2020].
#     start_date: Optional[str] = None,             # Optional start date "YYYY-MM-DD" (used with end_date).
#     end_date: Optional[str] = None,               # Optional end date   "YYYY-MM-DD" (used with start_date).
#     base_dir: str,                                # Model run directory; used for filename prefix and output folder structure.
#     figures_root: str,                            # Root output folder. A module subfolder (e.g., /timeseries/) is added automatically.
#     groups: Optional[Dict[str, Any]] = None,      # Composite definitions so you can request semantic variables:
#                                                   #   "chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"
#                                                   #   "phyto": ["P1_c","P2_c","P4_c","P5_c"]   (elementwise sum)
#     linewidth: float = 1.5,                       # Line thickness.
#     figsize: tuple = (10, 4),                     # Figure size in inches (width, height).
#     dpi: int = 150,                               # PNG resolution.
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional per-series styles, e.g. line colors:
#                                                   #   {"chl": {"line_color": "lightgreen"}, "temp": {"line_color": "lightblue"}}
#     verbose: bool = True,                         # Print progress (mask details, time window, saved path, etc.).
#     combine_by: Optional[str] = None,             # Multi-line modes for convenience:
#                                                   #   None       -> one PNG per (region × variable)  [default]
#                                                   #   "var"      -> one PNG per region,   lines = variables
#                                                   #   "region"   -> one PNG per variable, lines = regions
# ) -> None:
#     """
#     Plot regional mean time series using polygon masks and save PNG(s).
#
#     How masking works:
#       • A node mask is built from the shapefile/CSV polygon (nodes inside are kept).
#       • If mesh connectivity `nv` exists, an element mask can be derived (keep elements whose 3 nodes are inside).
#       • Area weighting is used automatically if `art1` is available; otherwise means are unweighted.
#       • Absolute-depth requests (e.g., depth=-8.0) are applied AFTER masking to ensure the correct local water column.
#
#     Output name pattern:
#       <prefix>__Region-<Name>__<VarOrMulti>__<DepthTag>__<TimeLabel>__Timeseries[__CombinedByVar|__CombinedByRegion].png
#
# Notes:
# - Region masks are built on the FVCOM grid; elements/nodes strictly inside the polygon are included.
# - If mesh connectivity `nv` is present, “strict” element-inclusion may be used (all three nodes inside).
# - If an area field (e.g., 'art1') exists, regional means are area-weighted; otherwise unweighted.
# - CSV boundaries should trace the polygon perimeter (or set convex_hull=True to wrap scattered points).
# - Works with Dask-chunked datasets; computation is triggered during reduction/plot.
# - Returns None; to view in a notebook, display the saved PNGs afterwards (e.g., with a gallery cell).


#Example — bottom zooplankton in first region (e.g., "Central"), full span
fig = region_timeseries(
    ds=ds,
    variables=["zoo"],
    regions=[REGIONS[0]],    # e.g., ("Central", {"shapefile": "...shp"})
    depth="bottom",
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)

# Example - Central region — surface, Jul 2018: chl + phyto + zoo on one plot
fig = region_timeseries(
    ds=ds,
    variables=["chl", "phyto", "zoo"],
    regions=[REGIONS[0]],                        # e.g., ("Central", {...})
    depth="surface",
    months=[7], years=[2018],
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS, styles=PLOT_STYLES,
    combine_by="var",
    verbose=False,
)

# Example -  Compare regions — bottom DOC, Apr–Oct 2018: one plot, one line per region
fig = region_timeseries(
    ds=ds,
    variables=["DOC"],
    regions=REGIONS,                             # multiple regions
    depth="bottom",
    years=[2018], months=[4,5,6,7,8,9,10],
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS, styles=PLOT_STYLES,
    combine_by="region",
    verbose=False,
)

# 4) Domain-wide Three-panel Figures (Surface ±1σ, Bottom ±1σ, Profile mean ±1σ) (1 figure with 3 subplots)
# Full argument reference for domain_three_panel(...)
# Each parameter below is annotated with what it does and accepted values.
# Produces a 3×1 (or similar) figure with:
#   • Surface time series ±1σ
#   • Bottom  time series ±1σ
#   • Vertical-profile mean (depth-avg) time series ±1σ
# One figure per variable; saves to disk; returns None.

# def domain_three_panel(
#     ds: xr.Dataset,                              # Xarray Dataset with FVCOM–ERSEM output (already opened/combined)
#     variables: list[str],                        # One or more names: native vars (e.g., "temp") or composites (e.g., "chl") if provided in `groups`
#     *,                                           # Everything after this must be passed as keyword-only (safer, clearer)
#     months=None,                                 # Calendar months to include (1–12) across all years; e.g., [7] or [4,5,6,7,8,9,10]; None = no month filter
#     years=None,                                  # Calendar years to include; e.g., [2018] or [2018, 2019]; None = no year filter
#     start_date=None,                             # Inclusive start date "YYYY-MM-DD"; used with end_date; None = no start bound
#     end_date=None,                               # Inclusive end date   "YYYY-MM-DD"; used with start_date; None = no end bound
#     base_dir: str,                               # Path to the model run folder; used for output subfolder and filename prefix
#     figures_root: str,                           # Root directory where figures are saved (module subfolder is created under this)
#     groups: Optional[Dict[str, Any]] = None,     # Composite definitions to allow semantic names in `variables`:
#                                                  #   {"chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"}    # string expression evaluated in ds namespace
#                                                  #   {"phyto": ["P1_c", "P2_c", "P4_c", "P5_c"]}     # list/tuple summed elementwise
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional per-variable style hints (e.g., line colors/labels used across panels)
#     dpi: int = 150,                              # Output resolution (dots per inch) for the saved PNG
#     figsize: tuple = (11, 9),                    # Figure size in inches (width, height)
#     verbose: bool = False,                       # If True, print progress (time window, file path, etc.)
# ) -> None:
#     pass  # Function computes domain-wide surface/bottom/depth-avg series (+/- 1σ), plots 3 panels, SAVES a PNG; returns None

# Output path pattern (per variable):
#   <figures_root>/<basename(base_dir)>/timeseries/
#     <prefix>__Domain__<VarOrGroup>__ThreePanel__<TimeLabel>__Timeseries.png
#
# where:
#   <prefix>    = file_prefix(base_dir)
#   <TimeLabel> = derived from months/years/start_date/end_date (AllTime, Jul, 2018, 2018-04–2018-10, ...)
#
# Notes:
# - Each panel shows the mean line and a ±1 standard deviation envelope for the selected vertical slice
#   (top panel = surface, middle = bottom, bottom = depth-averaged).
# - Spatial mean is over the full domain; if an area field (e.g., 'art1') exists, area weighting is applied.
# - Composites in `groups` let you pass semantic variables like "chl"/"phyto"/"zoo" without rewriting expressions.
# - Works with Dask-chunked datasets; computation occurs during reductions/plotting.
# - Returns None; to view in a notebook, display saved PNGs afterwards (e.g., using a small gallery cell).

# Example:  Domain three-panel — DOC (full run)
fig = domain_three_panel(
    ds=ds,
    variables=["DOC"],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)

#5) Station-specific Three-panel Figures (Surface ±1σ, Bottom ±1σ, Profile mean ±1σ) (1 figure with 3 subplots)
# Full argument reference for station_three_panel(...)
# Each parameter below is annotated with what it does and accepted values.
# Produces a 3×1 figure per (station × variable) with:
#   • Surface time series ±1σ (temporal σ at the station's nearest node)
#   • Bottom  time series ±1σ (temporal σ at the station's nearest node)
#   • Depth-averaged time series ±1σ (temporal σ at the station's nearest node)
# Saves one PNG per (station × variable); returns None.

# def station_three_panel(
#     ds: xr.Dataset,                               # Xarray Dataset with FVCOM–ERSEM output (already opened/combined)
#     variables: list[str],                         # One or more names: native vars (e.g., "temp") or composites (e.g., "chl") if provided in `groups`
#     stations: List[Tuple[str, float, float]],     # Station metadata as (name, lat, lon) in WGS84 decimal degrees
#                                                   #   - lon west of Greenwich is negative (e.g., -83.10)
#                                                   #   - nearest model *node* is selected by great-circle distance (WGS84)
#     *,                                            # Everything after this must be passed as keyword-only (safer, clearer)
#     months=None,                                  # Calendar months to include (1–12) across all years; e.g., [7] or [4,5,6,7,8,9,10]; None = no month filter
#     years=None,                                   # Calendar years to include; e.g., [2018] or [2018, 2019]; None = no year filter
#     start_date=None,                              # Inclusive start date "YYYY-MM-DD"; used with end_date; None = no start bound
#     end_date=None,                                # Inclusive end date   "YYYY-MM-DD"; used with start_date; None = no end bound
#     base_dir: str,                                # Path to the model run folder; used for output subfolder and filename prefix
#     figures_root: str,                            # Root directory where figures are saved (module subfolder is created under this)
#     groups: Optional[Dict[str, Any]] = None,      # Composite definitions to allow semantic names in `variables`:
#                                                   #   {"chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"}    # string expression evaluated in ds namespace
#                                                   #   {"phyto": ["P1_c", "P2_c", "P4_c", "P5_c"]}     # list/tuple summed elementwise
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional per-variable style hints (e.g., line colors/labels used across panels)
#     dpi: int = 150,                               # Output resolution (dots per inch) for the saved PNG
#     figsize: tuple = (11, 9),                     # Figure size in inches (width, height)
#     verbose: bool = False,                        # If True, print progress (resolved station index, time window, file path, etc.)
# ) -> None:
#     pass  # Function extracts nearest-node series per station, computes surface/bottom/depth-avg series, plots 3 panels with temporal ±1σ, SAVES PNG(s); returns None

# Output path pattern (per station × variable):
#   <figures_root>/<basename(base_dir)>/timeseries/
#     <prefix>__Station-<Name>__<VarOrGroup>__ThreePanel__<TimeLabel>__Timeseries.png
#
# where:
#   <prefix>    = file_prefix(base_dir)
#   <Name>      = station name from `stations`
#   <TimeLabel> = derived from months/years/start_date/end_date (AllTime, Jul, 2018, 2018-04–2018-10, ...)
#
# Notes:
# - σ shading is *temporal* at stations (single grid node): the envelope reflects time-wise standard deviation around the mean line.
# - Surface/bottom selections use the top/bottom sigma layers at the resolved nearest node; depth-avg is a vertical mean at that node.
# - Composites in `groups` let you pass semantic variables like "chl"/"phyto"/"zoo" without rewriting expressions.
# - Works with Dask-chunked datasets; computation occurs during reductions/plotting.
# - Returns None; to view in a notebook, display the saved PNGs afterwards (e.g., using a small gallery cell).

# Example: Station three-panel — chl at first station (full run)
fig = station_three_panel(
    ds=ds,
    variables=["chl"],
    stations=[STATIONS[0]],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)


# 6) Region three-panel 
# Full argument reference for region_three_panel(...)
# Each parameter below is annotated with what it does and accepted values.
# Produces a 3×1 figure per (region × variable) with:
#   • Surface time series ±1σ (SPATIAL σ across the region at each timestep)
#   • Bottom  time series ±1σ (SPATIAL σ across the region at each timestep)
#   • Depth-averaged time series ±1σ (SPATIAL σ across the region at each timestep)
# Saves one PNG per (region × variable); returns None.

# def region_three_panel(
#     ds: xr.Dataset,                               # Xarray Dataset with FVCOM–ERSEM output (already opened/combined)
#     variables: List[str],                         # One or more names: native vars (e.g., "temp") or composites (e.g., "chl") if provided in `groups`
#     regions: List[Tuple[str, Dict[str, Any]]],    # List of region specs as (region_name, spec_dict).
#                                                   #   spec_dict provides exactly ONE polygon source:
#                                                   #     {"shapefile": "/path/to/region.shp"}                       # optional filtering:
#                                                   #       + "name_field": "<FIELD>", "name_equals": "<VALUE>"
#                                                   #     {"csv_boundary": "/path/to/boundary.csv"}                  # CSV boundary polygon
#                                                   #       + "lon_col": "lon", "lat_col": "lat"                      # column names (defaults: lon/lat)
#                                                   #       + "convex_hull": True|False                               # wrap scattered points
#                                                   #       + "sort": "auto" | None                                   # attempt to order perimeter points
#     *,                                            # Everything after this must be passed as keyword-only (safer, clearer)
#     months=None,                                  # Calendar months to include (1–12) across all years; e.g., [7] or [4,5,6,7,8,9,10]; None = no month filter
#     years=None,                                   # Calendar years to include; e.g., [2018] or [2018, 2019]; None = no year filter
#     start_date=None,                              # Inclusive start date "YYYY-MM-DD"; used with end_date; None = no start bound
#     end_date=None,                                # Inclusive end date   "YYYY-MM-DD"; used with start_date; None = no end bound
#     base_dir: str,                                # Path to the model run folder; used for output subfolder and filename prefix
#     figures_root: str,                            # Root directory where figures are saved (module subfolder is created under this)
#     groups: Optional[Dict[str, Any]] = None,      # Composite definitions to allow semantic names in `variables`:
#                                                   #   {"chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"}    # string expression evaluated in ds namespace
#                                                   #   {"phyto": ["P1_c", "P2_c", "P4_c", "P5_c"]}     # list/tuple summed elementwise
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional per-variable style hints (e.g., line colors/labels used across panels)
#     dpi: int = 150,                               # Output resolution (dots per inch) for the saved PNG
#     figsize: tuple = (11, 9),                     # Figure size in inches (width, height)
#     verbose: bool = False,                        # If True, print progress (masking details, time window, file path, etc.)
# ) -> None:
#     pass  # Function masks nodes/elements inside each region, computes regional surface/bottom/depth-avg means,
#           # plots 3 panels with SPATIAL ±1σ envelopes per timestep, SAVES PNG(s); returns None

# Output path pattern (per region × variable):
#   <figures_root>/<basename(base_dir)>/timeseries/
#     <prefix>__Region-<Name>__<VarOrGroup>__ThreePanel__<TimeLabel>__Timeseries.png
#
# where:
#   <prefix>    = file_prefix(base_dir)
#   <Name>      = region name from `regions`
#   <TimeLabel> = derived from months/years/start_date/end_date (AllTime, Jul, 2018, 2018-04–2018-10, ...)
#
# Notes:
# - Region mask is built on the FVCOM grid; only nodes/elements inside the polygon are included.
# - If mesh connectivity `nv` is present, a strict element-inclusion rule (all three nodes inside) may be applied.
# - If an area field (e.g., 'art1') exists, regional means are area-weighted; otherwise unweighted.
# - The shaded ±1σ is **spatial** (spread across grid cells within the region at each time), unlike station_three_panel which uses **temporal** σ.
# - Composites in `groups` let you pass semantic variables like "chl"/"phyto"/"zoo" without rewriting expressions.
# - Works with Dask-chunked datasets; computation occurs during reductions/plotting.
# - Returns None; to view in a notebook, display the saved PNGs afterwards (e.g., with a small gallery cell).

#Example: Region three-panel — DOC in first region, Apr–Oct
fig = region_three_panel(
    ds=ds,
    variables=["DOC"],
    regions=[REGIONS[0]],
    months=[4,5,6,7,8,9,10],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)


# 7) Depth selection shorthand demos (sigma index, sigma value, absolute depth)

# Example: Domain — DOC at sigma layer index k=5, July
fig = domain_mean_timeseries(
    ds=ds,
    variables=["DOC"],
    depth=5,                  # == ("siglay_index", 5)
    months=[7],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)


# Example: Station — chl at sigma value s = -0.7 (in [-1, 0]), full run
fig = station_timeseries(
    ds=ds,
    variables=["chl"],
    stations=[STATIONS[0]],
    depth=-0.7,               # == ("sigma", -0.7)
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)


# Example: Region — temperature at absolute depth z = -8 m, Apr–Oct 2018
fig = region_timeseries(
    ds=ds,
    variables=["temp"],
    regions=[REGIONS[0]],
    depth=-8.0,               # == ("z_m", -8.0)  (meters; negative = below surface)
    years=[2018],
    months=[4,5,6,7,8,9,10],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    dpi=150,
    verbose=False,
)


print(" Timeseries examples completed. Figures saved under:", FIG_DIR)



 Timeseries examples completed. Figures saved under: /data/proteus1/scratch/moja/projects/Lake_Erie/fviz-plots/


In [ ]:
# Show saved figures for this run (timeseries)

from pathlib import Path
from IPython.display import display, Image, SVG

# Build the output root from your existing config
RUN_ROOT = Path(FIG_DIR) / Path(BASE_DIR).name     # e.g. <FIG_DIR>/<basename(BASE_DIR)>
OUT_ROOT = RUN_ROOT / "timeseries"                 

print("Looking under:", OUT_ROOT.resolve())

if not OUT_ROOT.exists():
    print(f" Folder does not exist: {OUT_ROOT}")
else:
    # grab newest first; include PNG and SVG
    files = sorted(
        list(OUT_ROOT.rglob("*.png")) + list(OUT_ROOT.rglob("*.svg")),
        key=lambda p: p.stat().st_mtime
    )
    if not files:
        print(f"No images found under {OUT_ROOT}")
    else:
        N = 20  # how many to show
        print(f"Found {len(files)} image(s). Showing the latest {min(N, len(files))}…")
        for p in files[-N:]:
            print("•", p.relative_to(RUN_ROOT))
            if p.suffix.lower() == ".svg":
                display(SVG(filename=str(p)))
            else:
                display(Image(filename=str(p)))
